Code from A3 problem1-prompt.ipynb and https://github.com/SewoongLee/reproduce-llama3-arithmetic/blob/main/llama3-tutorial-gsm8k.ipynb

You need to set your OPENAI_API_KEY environment variable in .env

In [1]:
from openai import OpenAI
from tqdm import tqdm
import textwrap
import dotenv
import os
%load_ext dotenv
%dotenv

client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# we use gpt-4o-mini as default: current pricing is 
# $0.150 / 1M input tokens, $0.600 / 1M output tokens
def generate_gpt_response(prompt,messages=[], model="gpt-3.5-turbo", temperature=0.7):
    messages.append({"role": "user", "content": prompt})
            
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = temperature
    )
    content=response.choices[0].message.content
    messages.append({'role': 'assistant', 'content': content})
    return messages


def print_message(message, width=80):
    text=message["role"]+": " + message["content"]
    wrapped_text = textwrap.fill(text, width=width)
    
    for line in wrapped_text.split('\n'):
        print(f" {line.ljust(width)} ")        
    

cannot find .env file


load gsm8k evolved dataset

In [2]:
import json

with open('gsm_evolved/output/gsm_evolved_20.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

for i in range(len(data)):
    print('Q:')
    print(data[i]['problem'])
    print()
# print('A:')
# print(train_data[0]['answer'])


Q:
Toby Fogel sold 520 kitchen gadgets in March, 2013 at a historical reenactment fair in Dubuque County, Iowa. In April, they sold 325% of the amount sold in the previous month. How many kitchen gadgets did Toby Fogel sell in total during March and April?

Q:
Reed Metz earns $15 an hour for dresses on Thursday at a planetarium gift shop in Greeley County, Nebraska. Yesterday, they did 73 minutes of dresses. How much did Reed Metz earn?

Q:
Tricia Laura is saving money to buy a camera which costs $4065. Tricia Laura has already saved $842. Their friend decided to give them $421 for this purpose, and their neighbor, twice as much as the friend. How much more money does Tricia Laura need to buy the camera?

Q:
Maximo Funez started reading a manual at a bus terminal store in Spartanburg County, South Carolina on Tuesday. They read 160 pages on the first day. On the second day, they read 110% more pages than the first day. How many pages did Maximo Funez read in total during the first two 

We will use the following system prompt

In [3]:
system_prompt= \
'''You are an expert math tutor specializing in grade-school-level problems like those in the GSM8K dataset. 
Solve each problem step by step, explaining the reasoning and calculations clearly and concisely to help the student understand the process. 
Please return your final answer after the delimiter \"####\" as a numeric number'''

We can answer from gpt api using fewshots in context learning and system prompt

In [4]:
import re

def extract_ans_from_response(answer: str, eos=None):
    if eos:
        answer = answer.split(eos)[0].strip()

    answer = answer.split('####')
    
    if len(answer) > 2:
        answer = answer[-2].strip()
    elif len(answer) == 2:
        answer = answer[-1].strip()
    else:
        answer = "NA"

    if isinstance(answer, str):
        for remove_char in [',', '$', '%', 'g']:
            answer = answer.replace(remove_char, '')

    try:
        return int(answer)
    except ValueError:
        return answer

# def get_num_answer(text):
#     num_matches = re.findall(r'\d+\.?\d*', text)

#     if not num_matches:
#         return None
    
#     final_answer = num_matches[-1]

#     return final_answer

# Implement the function answer_from_gpt(question,temperature,num_shots), which should return the gpt answer as a string
# (see how it is called in the function evaluate_fewshots below)
# temperature is the temperature parameter passed to gpt api
# if num_shots>0, you should use the first num_shots examples in train_data for in-context learning
# Please use the system prompt provided above (see lecture10-gpt.ipynb for examples of system prompt usage)
#
def answer_from_gpt(question, temperature, num_shots):
    example = []

    # if num_shots > 0:
    #     for i in range(num_shots):
    #         curr_ex = data[i]
    #         example.append({"role": "user", "content": curr_ex['question']})
    #         example.append({"role": "assistant", "content": curr_ex['answer']})

    full_prompt = system_prompt + "\n" + question
    # print("Q: ", full_prompt)

    messages = generate_gpt_response(prompt=full_prompt, messages=example, temperature=temperature)
    answer = messages[-1]["content"]
    # print("A: ", answer)
    last_token = extract_ans_from_response(answer)
    return last_token

Evaluate on a small subset of gsm8k test data

In [ ]:
repeats = 100

def evaluate_fewshots(temperature,num_shots):
    total_inaccurate = 0
    for k in tqdm(range(len(data))):
        print("Question ", k)
        num_corrects = 0
        for i in range(repeats):
            gpt_answer=answer_from_gpt(data[k]['problem'],temperature=temperature,num_shots=num_shots)
            true_answer= data[k]['result']
            # print(f'k={i}: gpt_answer={gpt_answer}  true_answer={true_answer}')   # this is for debug only
            if gpt_answer != "NA":
                try:
                    if float(gpt_answer) == float(true_answer):
                        # print("CORRECT")
                        num_corrects += 1
                    else:
                        # print("INCORRECT")
                        num_corrects += 0
                except ValueError:
                    # print("INCORRECT")
                    num_corrects += 0
        accuracy = num_corrects/repeats
        print(f'accuracy={accuracy:0.2f}')
        print()

        if accuracy < 1:
            total_inaccurate += 1

    print("Total Inaccurate: ", total_inaccurate)
        

temperature = 0
num_shots = 0

print(f'=== temperature={temperature} num_shots={num_shots} ===')
evaluate_fewshots(temperature,num_shots)

=== temperature=0 num_shots=0 ===


  0%|          | 0/20 [00:00<?, ?it/s]

Question  0
k=0: gpt_answer=2210  true_answer=2210
k=1: gpt_answer=1170  true_answer=2210
k=2: gpt_answer=1170.  true_answer=2210


  0%|          | 0/20 [00:05<?, ?it/s]


KeyboardInterrupt: 